# get a series masks

In [1]:
import gdal

one_vhr_path = "E:\\code\\Py_workplace\\satellite_flooding\\Data\\Germany\\Post\\20210718_10500500E6DD3C00.tif"
# Open .tif
dataset = gdal.Open(one_vhr_path)

# get the geo range
geotransform = dataset.GetGeoTransform()
left = geotransform[0]
top = geotransform[3]
right = left + geotransform[1] * dataset.RasterXSize
bottom = top + geotransform[5] * dataset.RasterYSize


In [4]:
dataset.GetDriver()


<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x0000018D19B1C6C0> >

In [5]:
dataset.GetProjectionRef()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [9]:
# get_masks_to_cover(given_region, mask_size):
''' for a given region, return a serious little masks to cover the region.
Input: given_region (xx), mask_size (km)

'''
# determine the parameters for the masks
# SetProjection



import numpy

gdal.AllRegister()
driver = gdal.GetDriverByName('MEM')  # just a varible, if want to save the .tif, set as "GTiff"

xsize = 200 # 
ysize = 200 # 输出影像的行数，根据尺寸和分辨率计算
bands = 1 # 输出影像的波段数，这里只有一个mask
datatype = gdal.GDT_Byte # 输出影像的数据类型，这里使用Byte
output_ds = driver.Create('', xsize, ysize, bands, datatype)

geotransform = [120, 0.0001, 0, 30, 0, -0.0001] # 地理变换参数，根据经纬度和分辨率计算
output_ds.SetGeoTransform(geotransform)


projection = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]' # 投影信息，这里使用WKT格式
output_ds.SetProjection(projection)

mask = numpy.ones((ysize, xsize), dtype=numpy.uint8) # 创建一个全是1的数组，大小和数据类型与输出影像一致
output_ds.GetRasterBand(1).WriteArray(mask) # 写入输出影像的第一个波段
# output_ds.FlushCache() # 保存输出影像

# 


0

In [ ]:

# define the masks

driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', preim_ncols, preim_nrows, 1, gdal.GDT_Byte)
# target_ds.SetGeoTransform((xmin, xres, 0, ymax, 0, yres))
# band = target_ds.GetRasterBand(1)
# target_ds.SetProjection(raster_srs.ExportToWkt())

# # Rasterize flooded roads
# gdal.RasterizeLayer(target_ds, [1], buffered_roads_layer, options=['ATTRIBUTE=mask_id'])
# # get binary road array (0 no-road, 1 road)
# road_flood_arr = target_ds.ReadAsArray()
# road_binary_arr = np.zeros(road_flood_arr.shape)
# road_binary_arr[road_flood_arr > 0.5] = 1
# road_arr_sum = np.sum(road_binary_arr)

In [1]:
import gdal
import numpy as np
mask_size = 2
given_region = [6.691455797, 50.588763568, 7.084461410, 50.296027904]

''' for a given region, return a serious smaller masks to cover the region.
Input: given_region (xx), mask_size (km)
Output: a serious of [left, top, right, bottom]
'''
# initialization
masks_list = []
reso = 0.0001  # the resolution, about 10 meters
reso_meter = reso * 100000


# parameters for masks
xsize = mask_size * 1000 / reso_meter * reso
ysize = mask_size * 1000 / reso_meter * reso


longitudes = np.arange(given_region[0], given_region[2], xsize, dtype = float).tolist()
latitudes = np.arange(given_region[1], given_region[3], -ysize, dtype = float).tolist()


for lon in longitudes:
    for lat in latitudes:
        left = lon
        top = lat
        right = left + xsize
        bottom = top - ysize
        masks_list.append([left, top, right, bottom])

# Combine the building and road masks

With the help of QGIS

# Cut satellite images and building, road, flooding masks

In [25]:

left, top, right, bottom = masks_list[1]

# left = 6.970763
# top = 50.510766
# right = left+0.002
# bottom = top-0.002
print(masks_list[1])

[6.691455797, 50.568763567999994, 6.711455796999999, 50.54876356799999]


In [28]:

one_vhr_path = "E:\\code\\Py_workplace\\satellite_flooding\\Data\\Germany\\Post\\20210718_10500500E6DD3C00.tif"

input_ds = gdal.Open(one_vhr_path, gdal.GA_ReadOnly)
projection = input_ds.GetProjection() # 获取投影信息
# projection = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]' # 投影信息，这里使用WKT格式
# print(masks_list[1])

# ds = gdal.Open(one_vhr_path)
# ds = gdal.Translate('new.tif', ds, projWin = [-75.3, 5.5, -73.5, 3.7])
# ds = None


clip_ds = gdal.Translate('clip_ds.tif', input_ds, 
                         projWin=[left, top, right, bottom], 
                         xRes=0.0001, yRes=0.0001,
                         outputSRS=projection)


In [32]:
print(clip_ds.RasterXSize)
print(clip_ds.RasterYSize)
clip_ds.GetGeoTransform()

200
200


(6.691455796986715, 0.0001, 0.0, 50.56876415018625, 0.0, -0.0001)

In [33]:

input_ds = None # 关闭input_ds
clip_ds = None # 关闭clip_ds



# download modis

In [2]:
import ee
# ee.Initialize()
import geemap


In [ ]:

import modis_toolbox as mod_tol


In [10]:

mask_geo = masks_list[1]
date_range = ['2021-07-02','2021-07-03']
save_path = "E:\\code\\Py_workplace\\satellite_flooding\\"
filename = 'modis.tif'


In [5]:

left, top, right, bottom = mask_geo
# geometry, date_range = get_vhr_reg_date(one_vhr_path)
geometry = ee.Geometry.Polygon([[left, bottom],
                        [left, top],
                        [right, top],
                        [right, bottom]],proj="EPSG:4326") # the proj is determinted in QGIS


In [6]:

_, one_modis_ima, _ = mod_tol.get_modis_terra_aqua(geometry, ee.DateRange(date_range[0], date_range[1]))      


In [7]:

# for now, the band 7 and the ratio of b1 to b2 are selected
def b1b2_swir_clip(img):
        return img.select("swir","b1b2_ratio")

modis_swir_b1b2 = one_modis_ima.map(b1b2_swir_clip)


In [34]:

geemap.download_ee_image_collection(modis_swir_b1b2, 
                                    out_dir=save_path, 
                                    filenames=[filename],
                                    crs="EPSG:4326")

Total number of images: 1



modis.tif: |          | 0.00/400k (raw) [  0.0%] in 00:00 (eta:     ?)

In [37]:
one_modis_path = "E:\\code\\Py_workplace\\satellite_flooding\\modis.tif"

modis_ds = gdal.Open(one_modis_path, gdal.GA_ReadOnly)
projection = modis_ds.GetProjection() # 获取投影信息
# projection = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]' # 投影信息，这里使用WKT格式
# print(masks_list[1])

# ds = gdal.Open(one_vhr_path)
# ds = gdal.Translate('new.tif', ds, projWin = [-75.3, 5.5, -73.5, 3.7])
# ds = None


modis_ds_up = gdal.Translate('modis_ds.tif', modis_ds, 
                         projWin=[left, top, right, bottom], 
                         xRes=0.0001, yRes=0.0001,
                         outputSRS=projection)

print(modis_ds_up.RasterXSize)
print(modis_ds_up.RasterYSize)
modis_ds_up.GetGeoTransform()

modis_ds_up = None
modis_ds = None


200
200


# Identify the overlap region between the pre vhr and post vhr

In [41]:

# def identify_overlap(tif_path_1, tif_path_2):
#     '''identify the overlap region for two given .tif file
#     output: [left top right bottom]
#     '''

tif_path_1 = "E:\\code\\Py_workplace\\satellite_flooding\\Data\\Germany\\Pre\\20180701_104001003E791C00.tif"
tif_path_2 = "E:\\code\\Py_workplace\\satellite_flooding\\Data\\Germany\\Post\\20210718_10500500E6DD3C00.tif"

# one_vhr_path = "E:\\code\\Py_workplace\\satellite_flooding\\Data\\Germany\\Post\\20210718_10500500E6DD3C00.tif"
# Open .tif
tif_1 = gdal.Open(tif_path_1)
geotransform_1 = tif_1.GetGeoTransform()
left_1 = geotransform_1[0]
top_1 = geotransform_1[3]
right_1 = left_1 + geotransform_1[1] * tif_1.RasterXSize
bottom_1 = top_1 + geotransform_1[5] * tif_1.RasterYSize

tif_2 = gdal.Open(tif_path_2)
geotransform_2 = tif_2.GetGeoTransform()
left_2 = geotransform_2[0]
top_2 = geotransform_2[3]
right_2 = left_2 + geotransform_2[1] * tif_2.RasterXSize
bottom_2 = top_2 + geotransform_2[5] * tif_2.RasterYSize

left = max(left_1,left_2)
top = min(top_1,top_2)

right = min(right_1,right_2)
bottom = max(bottom_1,bottom_2)

print([left_1, top_1, right_1, bottom_1])
print([left_2, top_2, right_2, bottom_2])
print([left, top, right, bottom])
print((left<right) and (top>bottom))

[6.687608736234032, 51.027255387440256, 6.9046901500298095, 49.9713756553876]
[6.691455796986715, 50.58876356793541, 7.084461409524862, 50.29602790441625]
[6.691455796986715, 50.58876356793541, 6.9046901500298095, 50.29602790441625]
True
